<a href="https://colab.research.google.com/github/zihaowang678/11785-25Spring-Project/blob/main/ChatGPT-4o-mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requirement Install

In [ ]:
!pip install openai
!pip install tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = "/content/drive/My Drive/DL Proj/data/"

Mounted at /content/drive


GPT4o-mini demo

In [ ]:
from openai import OpenAI
api_key = ''
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "Hello, which model are you?"}]
)

print(response.choices[0].message.content)


I'm based on OpenAI's GPT-3 model and trained on data up until October 2021. Please let me know how I can assist you!


Instruction Test

In [ ]:
import tiktoken
def num_tokens_from_messages(messages, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0

    for message in messages:
        num_tokens += len(encoding.encode(message["content"]))
        num_tokens += 4

    num_tokens += 2

    return num_tokens

In [ ]:
initial_instruction = "You are an expert in stock analysis which is focus on earning call analysis. An earnings call is a teleconference, or webcast, in which a public company discusses the financial results of a reporting period. I need you to analysis following earning call text in order to do sentiment analysis. Your output will only be on word: postive, negative or neutral."
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": initial_instruction}]
)

print(response.choices[0].message.content)

Sure, please provide the earnings call text you would like me to analyze for sentiment.


In [ ]:
from time import sleep
from tqdm import tqdm
import numpy as np
question = "Following is the one text, please only give one word output as positive, negative or neutral: "
gpt_4o_result = pd.read_csv(DATA_DIR + "gpt_4o_result.csv")
exist_transcript_id = gpt_4o_result["transcript_id"].tolist()
row = len(gpt_4o_result)
for i in ([1]):
  data = pd.read_parquet(DATA_DIR + f'batch_{i}_transcriptcomponent.parquet')
  transcript_id_list = sorted(list(data["transcriptid"].unique()))
  messages = {}
  for idx,transcript_id in enumerate(transcript_id_list):
    try:
      if transcript_id in exist_transcript_id:
        continue
      id_data = data[data["transcriptid"] == transcript_id]
      id_data = id_data.sort_values(by="componentorder")
      text = id_data["componenttext"].tolist()
      text = " ".join(text).replace("\"","\'")
      token_num = num_tokens_from_messages([{"role": "user", "content": question + text}])
      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[{"role": "user", "content": question + text}]
      )
      res = response.choices[0].message.content
      res = res.lower()
      print(res)
      if res == "positive" or res == "negative" or res == "neutral":
        gpt_4o_result.loc[row] = [transcript_id, res]
        print(f"batch {i}: {round(idx/len(transcript_id_list),3)} has finished, curr message has {token_num} tokens with answer: {res}")
      else:
        gpt_4o_result.loc[row] = [transcript_id, np.nan]
      sleep(1)
      row += 1
    except Exception as e:
      print(e)
      gpt_4o_result.loc[row] = [transcript_id, np.nan]
      row += 1


流式输出内容被截断，只能显示最后 5000 行内容。
positive
batch 1: 0.5 has finished, curr message has 9320 tokens with answer: positive
positive
batch 1: 0.5 has finished, curr message has 9342 tokens with answer: positive
positive
batch 1: 0.5 has finished, curr message has 14391 tokens with answer: positive
positive
batch 1: 0.501 has finished, curr message has 12672 tokens with answer: positive
neutral
batch 1: 0.501 has finished, curr message has 12655 tokens with answer: neutral
positive
batch 1: 0.501 has finished, curr message has 14366 tokens with answer: positive
positive
batch 1: 0.501 has finished, curr message has 9339 tokens with answer: positive
positive
batch 1: 0.501 has finished, curr message has 9339 tokens with answer: positive
positive
batch 1: 0.502 has finished, curr message has 14366 tokens with answer: positive
neutral
batch 1: 0.502 has finished, curr message has 12653 tokens with answer: neutral
positive
batch 1: 0.502 has finished, curr message has 10081 tokens with answer: positi

In [ ]:
gpt_4o_result.to_csv(DATA_DIR + "gpt_4o_result.csv", index=False)

In [ ]:
res = pd.read_csv(DATA_DIR + "gpt_4o_result.csv")
res

,transcript_id,sentiment
0,131,neutral
1,132,neutral
2,133,neutral
3,134,positive
4,290,neutral
...,...,...
9995,3366238,positive
9996,3367247,positive
9997,3367398,positive
9998,3367567,positive


In [ ]:
gpt_4o_result["sentiment"].value_counts()

,count
sentiment,
positive,6658
neutral,3178
negative,164
